In [25]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1809995,96.7,2973844,158.9,2973844,158.9
Vcells,3332434,25.5,8388608,64.0,8388608,64.0


In [51]:
varNames

[1] "x1" "x2" "x3"

In [65]:
# install.packages(c("gramEvol", "lightgbm", "data.table"))
library(gramEvol)
library(lightgbm)
library(data.table)

safeEval <- function(expr) {
  val <- try(eval(parse(text = expr), envir = df), silent = TRUE)
  if (inherits(val, "try-error") || any(is.nan(val)) || any(is.infinite(val))) {
    return(rep(NA, nrow(df)))
  }
  val
}

n = 0
# ------------------------------
# Example dataset
# ------------------------------
set.seed(42)
n <- 200
df <- data.table(
  x1 = runif(n, -5, 5),
  x2 = runif(n, -3, 3),
  x3 = runif(n, 0, 10)
)
df$y <- 3 * df$x1 - 2 * df$x2 + df$x3^2 + rnorm(n)

# ------------------------------
# Grammar definition
# ------------------------------
# Variables
varNames <- names(df)[names(df) != "y"]
num_generations <- 5
top_n <- 3
# Define grammar:
#   <expr> can be a variable or two expressions combined with + - * /
for (gen in 1:num_generations){
        
    grammarDef <- CreateGrammar(list(
      expr = gsrule("<var><op><var>","<var><op>"),
      op = gsrule("+", "-", "*","/","^"),    
      var  = gsrule("x1",'x2','x3')
    ))
    
    # ------------------------------
    # Fitness function
    # ------------------------------
    fitnessFunc <- function(exprStr) {
      print(exprStr)
      new_feature <- safeEval(exprStr)
      if (any(is.na(new_feature))) return(Inf)
      
      data_with_new <- as.matrix(cbind(df[, .(x1, x2, x3)], new_feature))
      label <- df$y
      
      dtrain <- lgb.Dataset(data_with_new, label = label)
      
      cv_result <- lgb.cv(
        params = list(objective = "regression", metric = "rmse", verbosity = -1),
        data = dtrain,
        nrounds = 30,
        nfold = 3,
        stratified = FALSE,
        verbose = -1
      )
      
      rmse_values <- unlist(cv_result$record_evals$valid$rmse$eval)
      min(rmse_values) # lower is better
    }
    
    
    
    set.seed(123)
    ge <- GrammaticalEvolution(
      grammarDef,
      fitnessFunc,
      terminationCost = 0.01,  # stop if RMSE < 0.01
      max.depth = 10,
      iterations = 20,
      popSize = 4
    )
  scores <- sapply(ge$population, fitnessFunc)
  ranked <- data.frame(expr = ge$population, score = scores)
  ranked <- ranked[order(ranked$score), ]
  
  cat("Best expression:", ranked$expr[1], "\nBest RMSE:", ranked$score[1], "\n")
  
  # Keep top N expressions as new variables for next gen
  best_exprs <- ranked$expr[1:top_n]
  varNames <- unique(c(varNames, paste0("(", best_exprs, ")")))
  
  # Add new columns to df for these expressions
  for (expr_str in best_exprs) {
    df[[expr_str]] <- safeEval(expr_str)
  }
    
}



<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
expression(x2^x2)
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1

Warning message in data.frame(expr = ge$population, score = scores):
“row names were found from a short variable and have been discarded”


Best expression: 
Best RMSE: 6.371993 
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
expression(x2^x2)
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <t

Warning message in data.frame(expr = ge$population, score = scores):
“row names were found from a short variable and have been discarded”


Best expression: 
Best RMSE: 6.371993 
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
expression(x2^x2)
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <t

...

Best expression: 
Best RMSE: 6.371993 
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
expression(x2^x2)
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <t

Warning message in data.frame(expr = ge$population, score = scores):
“row names were found from a short variable and have been discarded”


Best expression: 
Best RMSE: 6.371993 
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
expression(x2^x2)
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <text>:2:0: unexpected end of input
1: x1*
   ^>
<simpleError in parse(text = unescape.gt.lt(result.string)): <t

Warning message in data.frame(expr = ge$population, score = scores):
“row names were found from a short variable and have been discarded”


Best expression: 
Best RMSE: 6.371993 


In [62]:
best_exprs

NULL